Tana River Forest Classifier made by Kaiden Simon

In [3]:
import geemap
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Authenticate()
ee.Initialize()

tanariver = ee.FeatureCollection("projects/vpnww-299518/assets/tanariver")#Access via the assets tab in GEE(Table ID).
start_date="2018-01-01"#Set start_date(yy/mon/day)
end_date="2020-03-31"#Set End_date(yy/mon/day)
season = ee.Filter.date(start_date,end_date);#Filter image based on the time frame(start_date and end_date)

#-------------SENTINEL 1A(SAR) DATA------------------#
sentinel_1= ee.ImageCollection('COPERNICUS/S1_GRD');
sCollection=sentinel_1\
.filterBounds(tanariver) \
.filter(season) \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
.filter(ee.Filter.eq('instrumentMode', 'IW'))
desc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));
asc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));

composite = ee.Image.cat([
  asc.select('VH').mean(),
  asc.select('VV').mean(),
  desc.select('VH').mean()
]).focal_median();
composite.getInfo()

sentinel_2A = ee.ImageCollection('COPERNICUS/S2')\
.filterBounds(tanariver)\
.filterDate("2019-01-01","2020-01-01")\
.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",10))\
.median()\
.select('B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B10', 'B11')\
.clip(tanariver)#This Clips the imagery to our Study Area Extent

sentinel_2Avispar={"min":0, "max":2500,"bands": ['B4','B3','B2']}#Visualization parameters used.


Fused_images = sentinel_2A.addBands(composite)#Creation of composite image of sentinel 1 and sentinel 2A
#Visualization parameters for fused images.
fused_vis = {
        'min':0,
        'max'  : 2500,
        'bands' : ['B2', 'B3','B4']#Band selection.
    }

vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 5000.0, 'opacity': 1.0, 'gamma': 1.0}


#The classifier is based on the property with a given unique value i.e all class picked as forest_high has a given unique value (0).
training = ee.FeatureCollection('projects/vpnww-299518/assets/tanariverTrainingAsset1GEE')

#Create a buffer:
# training = training1.map(lambda f: f.buffer(5)) #This if you want the points/ Pplygons you use for classification to be in a 5M Buffer.
# Map.addLayer(training1,{},"buffer")
label = 'landcover'#Unique property(landcover) with different unique values..
bands=["B4","B3","B2","VV","VH"]
Input=Fused_images.select(bands)
trainImage=Input.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})

trainingData=trainImage.randomColumn()

trainSet=trainingData.filter(ee.Filter.lessThan('random',0.75))
testdata=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.75))#visualization parameters applied on the Random forest classifier
init_params = {"numberOfTrees":90, # the number of individual decision tree models
              "variablesPerSplit":None,  # the number of features to use per split
              "minLeafPopulation":1, # smallest sample size possible per leaf
              "bagFraction":0.5, # fraction of data to include for each individual tree model
              "maxNodes":None, # max number of leafs/nodes per tree
               "seed":0}  # random seed for "random" choices like sampling. Setting this allows others to reproduce your exact results even with stocastic parameters

classifier = ee.Classifier.smileRandomForest(**init_params).train(trainSet, label, bands)
# Classify the image.

#Application of the random forest classifier for the purpose of image classification.
classified=Input.classify(classifier)
palette = [
  'darkgreen', #forest_high(0)
  'lightgreen', #forest_medium(1)#90ee90
 ];
Map=geemap.Map()
Map.addLayer(classified,
             {'min': 0, 'max': 1, 'palette': palette},
             'classification')
Map.centerObject(tanariver, 14)
Map.addLayer(Fused_images,fused_vis,"Intergrated_image",False)

Map

Enter verification code: 4/1Adeu5BWcFwxyaSp2iWFCzR5LlmJ-O99kIZy2MB04YgX5oRQrhosinVTyLMI

Successfully saved authorization token.


Map(center=[-0.9518791821998106, 39.87609231323469], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
train_accuracy = classifier.confusionMatrix()
train_accuracy.getInfo()

[[118, 0], [1, 56]]

In [9]:
train_accuracy = classifier.confusionMatrix()
overall_accuracy= train_accuracy.accuracy()#Overall classification accuracy
overall_accuracy_perc=overall_accuracy.multiply(100)
overall_accuracy_tranc=ee.Number(overall_accuracy_perc).format('%.3f')
print('Overall Accuracy:',overall_accuracy_tranc.getInfo())

In [10]:
kappa_accuracy=train_accuracy.kappa().multiply(100)#Kappa accuracy
kappa_accuracy_tran=ee.Number(kappa_accuracy).format('%.3f')
print('Kappa Accuracy:',kappa_accuracy_tran.getInfo())

In [11]:
train_accuracy.producersAccuracy().getInfo(),("Producer_Accuracy")

In [12]:
#ESTIMATED CROP AREA
Total_studyArea= classified.geometry().area()
Total_AreaSqKm = ee.Number(Total_studyArea).divide(4046.86).round()
names = ['forest_high','forest_low','forest_medium','water','builtup','bareland','agriculture']

count = classified.eq([0,1,2,3,4,5,6])#

Estimated_area = count.multiply(ee.Image.pixelArea()).divide(4046.86).rename(names)#mutiply and divide the area with 1e6 to conv into km

crop_area = Estimated_area.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': tanariver,
  'scale': 30,
  'maxPixels': 1e9
})
# print(crop_area,"Crop Area in Acres:")
print("Estimated Class areas (Acres) ")

# Total_AreaSqKm.getInfo()
def trancuate(key, value):
    return ee.Number(value).format('%.3f')

areatran = crop_area.map(trancuate)
areatran.getInfo()